### instalar dependencias

In [103]:
# ! pip install pandas
#! pip install requests

### Obtener data

In [104]:
import requests 





url = "https://www.getonbrd.com/api/v0/search/jobs"
params = {
    "query": "Javadeveloper",
    "per_page": 2,
    "page": 1
}

response = requests.get(url, params=params)

raw_data = response.json()
print(raw_data)



{'data': [{'id': 'java-angular-fullstack-developer-ensitech-remote', 'type': 'job', 'attributes': {'title': 'Java Angular Full-Stack Developer', 'description_headline': 'Qualifications and requirements', 'description': '<ul><li>At least 8 years of experience in IT overall.</li><li>Strong proficiency in Java development, with 4+ years of hands-on experience.</li><li>2+ years of experience with TypeScript front-end frameworks, preferably Angular 8+.</li><li>Experience with Oracle and/or Postgres databases, including database design, query optimization, and interaction from an application developer perspective.</li><li>Familiarity with cloud deployment and CI/CD tools such as Jenkins.</li><li>Solid understanding of software development best practices, including TDD, design patterns, and clean code principles.</li><li>Excellent communication skills, both written and verbal.</li><li>Experience with agile methodologies such as Scrum or XP.</li><li>Ability to work independently and as part of

### Construir data frame

In [105]:
import pandas as pd

data = pd.read_csv("World-happiness-report-2024.csv")


df = pd.DataFrame(data)

### Clases

In [106]:
class Job:
    def __init__(self, id, remote, country, title, description, functions, published_at, min_salary, max_salary, modality, seniority, category_name, tags, perks, company):
        self.id = id
        self.remote = remote
        self.country = country
        self.title = title
        self.description = description
        self.functions = functions
        self.published_at = published_at 
        self.min_salary = min_salary
        self.max_salary = max_salary
        self.modality = modality
        self.seniority = seniority
        self.category_name = category_name
        self.tags = tags
        self.perks = perks
        self.company = company


class Company:
    def __init__(self, id, description, long_description, projects, benefits, country):
        self.id = id
        self.description = description
        self.country = country
        self.long_description = long_description 
        self.description = description
        self.projects= projects
        self.benefits = benefits
        self.country = country





### Parser

In [107]:
def parseCompany(raw):
    id= raw.id
    description = raw.description
    long_description = raw.long_description
    projects = raw.projects
    benefits = raw.benefits
    country = raw.country

    company = Company(id, description, long_description, projects, benefits, country)
    
    return company

def parseJob(raw):
    id = raw["id"]
    country = raw["country"]
    title = raw["title"]
    description = raw["description"]
    functions = raw["functions"]
    published_at = raw["published_at"]
    min_salary = raw["min_salary"]
    max_salary = raw["max_salary"]
    modality = raw["modality"]
    seniority = raw["seniority"]
    category_name = raw["category_namem"]
    tags = raw["tags"]
    perks = raw["perks"]
    company = raw["company"]

    job = Job(id, country, title, description, functions, published_at, min_salary, max_salary, modality, seniority, category_name, tags, perks, company)
    
    return job


### Transformar

In [108]:
print(raw_data)
jobList = raw_data["data"]
parsedJobList = []

for job in jobList:
    parsedJobList.append(parseJob(job))

print(parsedJobList)

{'data': [{'id': 'java-angular-fullstack-developer-ensitech-remote', 'type': 'job', 'attributes': {'title': 'Java Angular Full-Stack Developer', 'description_headline': 'Qualifications and requirements', 'description': '<ul><li>At least 8 years of experience in IT overall.</li><li>Strong proficiency in Java development, with 4+ years of hands-on experience.</li><li>2+ years of experience with TypeScript front-end frameworks, preferably Angular 8+.</li><li>Experience with Oracle and/or Postgres databases, including database design, query optimization, and interaction from an application developer perspective.</li><li>Familiarity with cloud deployment and CI/CD tools such as Jenkins.</li><li>Solid understanding of software development best practices, including TDD, design patterns, and clean code principles.</li><li>Excellent communication skills, both written and verbal.</li><li>Experience with agile methodologies such as Scrum or XP.</li><li>Ability to work independently and as part of

KeyError: 'country'